<a href="https://colab.research.google.com/github/teerdhalahari/derma-ai-tool/blob/main/field_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/apoorvjha/SkinDiseaseCV.git

Cloning into 'SkinDiseaseCV'...
remote: Enumerating objects: 18766, done.
remote: Counting objects: 100% (442/442), done.
remote: Compressing objects: 100% (289/289), done.
remote: Total 18766 (delta 134), reused 442 (delta 134), pack-reused 18324
Receiving objects: 100% (18766/18766), 1.61 GiB | 33.48 MiB/s, done.
Resolving deltas: 100% (136/136), done.
Updating files: 100% (19474/19474), done.


In [ ]:
import os
import numpy as np
from zipfile import ZipFile
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split


data_dir = '/content/SkinDiseaseCV/static/Dataset_Dermat'

img_width, img_height = 224, 224
batch_size = 64
epochs = 10

# Get the list of image paths and corresponding labels
image_paths = []
labels = []

for class_name in os.listdir(data_dir):
    class_path = os.path.join(data_dir, class_name)
    if os.path.isdir(class_path):
        for img_name in os.listdir(class_path):
            img_path = os.path.join(class_path, img_name)
            image_paths.append(img_path)
            labels.append(class_name)

# Map class names to integers
class_to_index = {class_name: index for index, class_name in enumerate(np.unique(labels))}
index_to_class = {index: class_name for class_name, index in class_to_index.items()}
labels = [class_to_index[label] for label in labels]

# Split the data into training and testing sets
train_paths, test_paths, train_labels, test_labels = train_test_split(image_paths, labels, test_size=0.2, random_state=42)

# Use a function to load and preprocess images from paths
def load_and_preprocess_image(img_path, label):
    img = load_img(img_path, target_size=(img_width, img_height))
    img_array = img_to_array(img) / 255.0  # Normalize pixel values to [0, 1]
    return img_array, label

# Create data generators
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='sparse',
    subset='training',
    shuffle=True,
    seed=42
)

validation_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='sparse',
    subset='validation',
    shuffle=False,
    seed=42
)

# Build the CNN model
num_classes = len(np.unique(labels))  # Number of classes in the dataset
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    class_weight=None  # Use class weights if needed
)

# Evaluate the model on the test data
test_generator = test_datagen.flow_from_directory(
    data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='sparse',
    shuffle=False
)

test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.samples // batch_size)
print(f"Test Accuracy: {test_acc}")

# Predict on a sample test batch
sample_images, sample_labels = next(test_generator)
predictions = model.predict(sample_images)

# Convert predictions to class indices
predicted_indices = np.argmax(predictions, axis=1)

# Map predicted indices to class names
# Map predicted indices to class names
predicted_labels = [index_to_class[index] for index in predicted_indices]

# Map true indices to class names for true labels
true_labels = [index_to_class[int(true_label)] for true_label in sample_labels]

print("Sample Predictions:")
for true_label, predicted_label in zip(true_labels, predicted_labels):
    print(f"True: {true_label}, Predicted: {predicted_label}")


In [ ]:
model.save('fpmodel_4')

In [ ]:
from google.colab import files
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Upload the new image file
uploaded = files.upload()

# Preprocess the new image
def preprocess_single_image(img_path):
    img = load_img(img_path, target_size=(img_width, img_height))
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    return img_array

# Load the trained model
model = tf.keras.models.load_model('/content/fpmodel_4')

# Make predictions on the new image
img_path = '/content/diseaseone.jpg'  # Replace with the actual path
preprocessed_img = preprocess_single_image(img_path)
predictions = model.predict(preprocessed_img)

# Map predicted index to class name
predicted_index = np.argmax(predictions)
predicted_class = index_to_class[predicted_index]

# Print the result
print(f"Predicted Class: {predicted_class}")


Saving diseaseone.jpg to diseaseone (1).jpg
1/1 [==============================] - 0s 307ms/step
Predicted Class: Eczema Photos
